In [2]:
import yt_dlp
import json
import os
from random import random
os.listdir(".")

['scott_psych.json',
 'yvette_election.m4a',
 '.DS_Store',
 'Speeches-Copy3.ipynb',
 'Speeches-Copy1.ipynb',
 'Tweets.ipynb',
 'scott_interview.m4a',
 'psych.m4a',
 'Speeches-Copy4.ipynb',
 'Speeches.ipynb',
 'Yvette_Trans.json',
 'Speeches-Copy2.ipynb',
 'Yvette_election.json',
 '.ipynb_checkpoints',
 'psych_scott.json',
 'yvette_interview.json',
 'testin.ipynb',
 'scott_interview.json',
 'scott.json',
 'yvette_trans.m4a']

# Utils

In [3]:
# OpenAI Embeddings
from openai import OpenAI
op = OpenAI(api_key=os.environ['OPENAI'])

def emb(text):
    response = op.embeddings.create(
        input=text,
        model="text-embedding-3-large"
    )

    return response.data[0].embedding


In [17]:
# YT DLP
ydl_opts = {
    'format': 'm4a/bestaudio/best',
    # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    'postprocessors': [{  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
    }]
}


In [4]:
# Whisper
import whisper
model = whisper.load_model("base")

In [5]:
os.listdir(".")

['scott_psych.json',
 'yvette_election.m4a',
 '.DS_Store',
 'Speeches-Copy3.ipynb',
 'Speeches-Copy1.ipynb',
 'Tweets.ipynb',
 'scott_interview.m4a',
 'psych.m4a',
 'Speeches-Copy4.ipynb',
 'Speeches.ipynb',
 'Yvette_Trans.json',
 'Speeches-Copy2.ipynb',
 'Yvette_election.json',
 '.ipynb_checkpoints',
 'psych_scott.json',
 'yvette_interview.json',
 'testin.ipynb',
 'scott_interview.json',
 'scott.json',
 'yvette_trans.m4a']

# Get Videos and Transcribe

In [91]:
videos = [
#     "https://www.youtube.com/watch?v=dbKyKpJHHaw",
# "https://www.youtube.com/watch?v=Ed6PLI06xtI",
# "https://www.youtube.com/watch?v=8c4G6VRyVsk",
"https://www.youtube.com/watch?v=xLveM3T_YRM"
         ]

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(videos)


[youtube] Extracting URL: https://www.youtube.com/watch?v=xLveM3T_YRM
[youtube] xLveM3T_YRM: Downloading webpage
[youtube] xLveM3T_YRM: Downloading ios player API JSON
[youtube] xLveM3T_YRM: Downloading android player API JSON


[youtube] xLveM3T_YRM: Downloading android player API JSON


[youtube] xLveM3T_YRM: Downloading android player API JSON


[youtube] xLveM3T_YRM: Downloading android player API JSON


[youtube] xLveM3T_YRM: Downloading m3u8 information
[info] xLveM3T_YRM: Downloading 1 format(s): 140
[download] Destination: Legalizing Psychedelics in San Francisco ｜ Interview with Sen. Scott Wiener ｜ DoubleBlind [xLveM3T_YRM].m4a
[download] 100% of   26.46MiB in 00:00:03 at 7.14MiB/s     
[FixupM4a] Correcting container of "Legalizing Psychedelics in San Francisco ｜ Interview with Sen. Scott Wiener ｜ DoubleBlind [xLveM3T_YRM].m4a"
[ExtractAudio] Not converting audio Legalizing Psychedelics in San Francisco ｜ Interview with Sen. Scott Wiener ｜ DoubleBlind [xLveM3T_YRM].m4a; file is already in target format m4a


In [6]:
transcript = model.transcribe(
    word_timestamps=True,
    audio="yvette_election.m4a",
    verbose=False
)

/opt/homebrew/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detected language: English


 99%|████████████████████████████████▋| 157214/158826 [17:31<00:10, 149.55frames/s]


# Compress

In [ ]:
import os
os.listdir(".")

In [7]:
name = "Yvette"
description = "an interview about the election"
intial_details = {
    "id": "https://www.youtube.com/watch?v=dbKyKpJHHaw",
    "author": name,
    "date": "Oct 31, 2022"
}

In [8]:
from tqdm import tqdm
import copy

# progress = 
def process_timestamps(transcript):
    time_chunks = []
    index = 0

    
    
    while index<len(transcript['segments']):
        section = copy.deepcopy(intial_details)
        text = transcript['segments'][index]['text']
        for i in range(4):
            try:
                text = text + transcript['segments'][index]['text']
                index+=1
            except:
                print('reached end')
        section['text'] = f"{name} said \"\"\"{text.strip()}\"\"\" in {description}"
        section['timestamp'] = transcript['segments'][index-1]['start']
        section['embedding'] = emb(section['text'])
        index+=1
        time_chunks.append(section)
    return time_chunks

chunks = process_timestamps(transcript)



In [10]:
file_save = open("yvette_election.json", 'w')
json.dump(chunks, file_save)